# 6 AutoGen - Multi agents

The AutoGen ecosystem provides everything you need to create AI agents, especially multi-agent workflows -- framework, developer tools, and applications.

The framework uses a layered and extensible design. Layers have clearly divided responsibilities and build on top of layers below. This design enables you to use the framework at different levels of abstraction, from high-level APIs to low-level components.

Core API implements message passing, event-driven agents, and local and distributed runtime for flexibility and power. It also support cross-language support for .NET and Python.
AgentChat API implements a simpler but opinionated API for rapid prototyping. This API is built on top of the Core API and is closest to what users of v0.2 are familiar with and supports common multi-agent patterns such as two-agent chat or group chats.
Extensions API enables first- and third-party extensions continuously expanding framework capabilities. It support specific implementation of LLM clients (e.g., OpenAI, AzureOpenAI), and capabilities such as code execution.

<img src="https://github.com/microsoft/autogen/raw/main/autogen-landing.jpg" width=400>

> https://microsoft.github.io/autogen/stable/<br>
> https://github.com/microsoft/autogen

In [1]:
import os
import sys

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.auth.azure import AzureTokenProvider
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from azure.identity import DefaultAzureCredential
from dotenv import load_dotenv

## Settings

In [2]:
sys.version

'3.10.14 (main, May  6 2024, 19:42:50) [GCC 11.2.0]'

In [3]:
load_dotenv("azure.env")

True

In [4]:
token_provider = AzureTokenProvider(
    DefaultAzureCredential(),
    "https://cognitiveservices.azure.com/.default",
)

In [5]:
client = AzureOpenAIChatCompletionClient(
    azure_deployment="gpt-4o-mini",
    model="gpt-4o-mini",
    api_version="2024-06-01",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    azure_ad_token_provider=token_provider,  # Optional if you choose key-based authentication.
)

## Agents definition

In [6]:
planner_agent = AssistantAgent(
    "planner_agent",
    model_client=client,
    description="A helpful assistant that can plan trips.",
    system_message="You are a, AI helpful assistant that can suggest a travel plan for a user based on their request.",
)

local_agent = AssistantAgent(
    "local_agent",
    model_client=client,
    description="A local assistant that can suggest local activities or places to visit.",
    system_message="You are an AI helpful assistant that can suggest authentic and interesting local activities or places to visit for a user and can utilize any context information provided.",
)

language_agent = AssistantAgent(
    "language_agent",
    model_client=client,
    description="A helpful assistant that can provide language tips for a given destination.",
    system_message="You are an AI helpful assistant that can review travel plans, providing feedback on important/critical tips about how best to address language or communication challenges for the given destination. If the plan already includes language tips, you can mention that the plan is satisfactory, with rationale.",
)

travel_summary_agent = AssistantAgent(
    "travel_summary_agent",
    model_client=client,
    description="A helpful assistant that can summarize the travel plan.",
    system_message="You are an AI helpful assistant that can take in all of the suggestions and advice from the other agents and provide a detailed final travel plan. You must ensure that the final plan is integrated and complete. YOUR FINAL RESPONSE MUST BE THE COMPLETE PLAN. When the plan is complete and all perspectives are integrated, you can respond with TERMINATE.",
)


In [7]:
planner_agent

In [8]:
local_agent

In [9]:
language_agent

In [10]:
travel_summary_agent

## Examples

In [11]:
query = "Plan a 10 days trip to France."

In [12]:
termination = TextMentionTermination("TERMINATE")

group_chat = RoundRobinGroupChat(
    [planner_agent, local_agent, language_agent, travel_summary_agent], termination_condition=termination
)

await Console(group_chat.run_stream(task=query))

---------- TextMessage (user) ----------
Plan a 10 days trip to France.
---------- TextMessage (planner_agent) ----------
Sure! Here’s a suggested 10-day itinerary for your trip to France, covering diverse regions, culture, cuisine, and landmarks.

### Day 1: Arrival in Paris
- **Morning:** Arrive in Paris. Check into your hotel.
- **Afternoon:** Visit the iconic Eiffel Tower. Take a leisurely walk along the Seine.
- **Evening:** Enjoy dinner in a bistro in the Latin Quarter. Consider visiting a cabaret show at the Moulin Rouge.

### Day 2: Explore Paris
- **Morning:** Visit the Louvre Museum. Don’t miss the Mona Lisa and the Venus de Milo.
- **Afternoon:** Stroll through the Tuileries Garden and head to Place de la Concorde.
- **Evening:** Explore Montmartre. Visit the Sacré-Cœur Basilica and enjoy the view of the city.

### Day 3: More of Paris
- **Morning:** Visit the Notre-Dame Cathedral (you can view the exterior and nearby Île de la Cité).
- **Afternoon:** Explore the Musée d'Ors

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Plan a 10 days trip to France.', type='TextMessage'), TextMessage(source='planner_agent', models_usage=RequestUsage(prompt_tokens=42, completion_tokens=802), metadata={}, content="Sure! Here’s a suggested 10-day itinerary for your trip to France, covering diverse regions, culture, cuisine, and landmarks.\n\n### Day 1: Arrival in Paris\n- **Morning:** Arrive in Paris. Check into your hotel.\n- **Afternoon:** Visit the iconic Eiffel Tower. Take a leisurely walk along the Seine.\n- **Evening:** Enjoy dinner in a bistro in the Latin Quarter. Consider visiting a cabaret show at the Moulin Rouge.\n\n### Day 2: Explore Paris\n- **Morning:** Visit the Louvre Museum. Don’t miss the Mona Lisa and the Venus de Milo.\n- **Afternoon:** Stroll through the Tuileries Garden and head to Place de la Concorde.\n- **Evening:** Explore Montmartre. Visit the Sacré-Cœur Basilica and enjoy the view of the city.\n\n### Day

In [13]:
query = "Plan a 5 days trip to Turin & Milan."

In [14]:
termination = TextMentionTermination("TERMINATE")

group_chat = RoundRobinGroupChat(
    [planner_agent, local_agent, language_agent, travel_summary_agent], termination_condition=termination
)

await Console(group_chat.run_stream(task=query))

---------- TextMessage (user) ----------
Plan a 5 days trip to Turin & Milan.
---------- TextMessage (planner_agent) ----------
Absolutely! Here's a suggested 5-day itinerary for your trip to Turin and Milan, two beautiful cities in Northern Italy known for their rich history, fashion, and delicious cuisine.

### Day 1: Arrival in Turin
- **Morning:** Arrive in Turin. Check into your hotel. 
- **Afternoon:** Visit the **Mole Antonelliana**, the iconic symbol of the city, and explore the **National Cinema Museum** inside.
- **Evening:** Stroll around **Piazza Vittorio Veneto** and enjoy dinner at a traditional trattoria. Try local dishes like agnolotti or bagna cauda.

### Day 2: Explore Turin
- **Morning:** Visit **Palazzo Reale** (Royal Palace of Turin) and the **Turin Cathedral**, home to the Shroud of Turin.
- **Afternoon:** Explore the **Egyptian Museum**, one of the most important of its kind in the world. 
- **Evening:** Walk along the **Po River** and enjoy an aperitivo in the Q

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Plan a 5 days trip to Turin & Milan.', type='TextMessage'), TextMessage(source='planner_agent', models_usage=RequestUsage(prompt_tokens=863, completion_tokens=639), metadata={}, content="Absolutely! Here's a suggested 5-day itinerary for your trip to Turin and Milan, two beautiful cities in Northern Italy known for their rich history, fashion, and delicious cuisine.\n\n### Day 1: Arrival in Turin\n- **Morning:** Arrive in Turin. Check into your hotel. \n- **Afternoon:** Visit the **Mole Antonelliana**, the iconic symbol of the city, and explore the **National Cinema Museum** inside.\n- **Evening:** Stroll around **Piazza Vittorio Veneto** and enjoy dinner at a traditional trattoria. Try local dishes like agnolotti or bagna cauda.\n\n### Day 2: Explore Turin\n- **Morning:** Visit **Palazzo Reale** (Royal Palace of Turin) and the **Turin Cathedral**, home to the Shroud of Turin.\n- **Afternoon:** Exp

In [17]:
query = "Plan a 1 day visit in Seattle. Add some emojis."

In [18]:
termination = TextMentionTermination("TERMINATE")

group_chat = RoundRobinGroupChat(
    [planner_agent, local_agent, language_agent, travel_summary_agent], termination_condition=termination
)

await Console(group_chat.run_stream(task=query))

---------- TextMessage (user) ----------
Plan a 1 day visit in Seattle. Add some emojis.
---------- TextMessage (planner_agent) ----------
Absolutely! Here’s a lively one-day itinerary for your visit to Seattle! 🌟☔

### Morning ☕🌅
- **Pike Place Market**: Start your day at the iconic **Pike Place Market**. Wander the stalls, watch the fishmongers throw fish, and grab a coffee from the original **Starbucks**! ☕🐟
- **Breakfast**: Enjoy a delicious breakfast at **Bacco Cafe** or try a pastry from **Pike Place Bakery**. 🍽️🥐

### Late Morning 🚶‍♂️🌊
- **Seattle Waterfront**: Stroll along the waterfront and stop by the **Seattle Aquarium** to see sea otters and other marine life. 🐠🌊
- **Seattle Great Wheel**: Experience breathtaking views on the **Seattle Great Wheel**. 🎡🌄

### Afternoon 🏙️🍣
- **Space Needle**: Head to the **Space Needle**. Enjoy panoramic views of the city and, if you’re up for it, test your courage on the glass floor! 🏔️😲
- **Lunch**: Grab lunch nearby at **The 85th Floor R

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Plan a 1 day visit in Seattle. Add some emojis.', type='TextMessage'), TextMessage(source='planner_agent', models_usage=RequestUsage(prompt_tokens=2074, completion_tokens=500), metadata={}, content='Absolutely! Here’s a lively one-day itinerary for your visit to Seattle! 🌟☔\n\n### Morning ☕🌅\n- **Pike Place Market**: Start your day at the iconic **Pike Place Market**. Wander the stalls, watch the fishmongers throw fish, and grab a coffee from the original **Starbucks**! ☕🐟\n- **Breakfast**: Enjoy a delicious breakfast at **Bacco Cafe** or try a pastry from **Pike Place Bakery**. 🍽️🥐\n\n### Late Morning 🚶\u200d♂️🌊\n- **Seattle Waterfront**: Stroll along the waterfront and stop by the **Seattle Aquarium** to see sea otters and other marine life. 🐠🌊\n- **Seattle Great Wheel**: Experience breathtaking views on the **Seattle Great Wheel**. 🎡🌄\n\n### Afternoon 🏙️🍣\n- **Space Needle**: Head to the **Space

In [19]:
# close client
await client.close()